<a id='Right-Scraping-and-cleaning'></a>

In [1]:
import requests
from tqdm import tqdm_notebook
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
#from langdetect import detect

In [2]:
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000

In [3]:
np.set_printoptions(threshold=np.inf)
pd.options.display.max_colwidth = 1000

- List of people I considered for this category

In [62]:
#List of name to scrape
lista = ['David Goodhart',
         'Ben Shapiro',
         'Jared Taylor', 
         'David Duke',
         'Matthew Goodwin',
         'Eric Kaufmann', 
         'Richard Bertrand',
         'Augustus Sol Invictus',
         'Peter Hitchens',
         'David Irving', 
         'Jason Kessler', 
         'Sebastian Gorka',
         'Tomislav Sunić', 
         'Paul Weyrich', 
         'Pat Buchanan', 
         'Steve Bannon', 
         'Ann Coulter', 
         'Milo Yiannopoulos', 
         'Vox Day',
         'Steve Sailer',
         'Stefan Molyneux',
         'Alex Jones', 
         'John Derbyshire', 
         'Mike Cernovich',
         'Peter Brimelow', 
         'Katie Hopkins', 
         'Laura Loomer', 
         'Paul Joseph',
         'Arthur Kemp', 
         'Tommy Robinson',
         'Raheem Kassam', 
         'Jerome Corsi', 
         'Daniel Drezner',
         'Lou Dobbs', 
         'Pamela Geller', 
         'Robert Spencer', 
         'Karl Hess', 
         'Hans-Hermann Hoppe',
         'William Luther Pierce', 
         'Joel Pollak', 
         'Matt Drudge',
         'Nicholas Wade', 
         'John R. Bolton', 
         "Dinesh D'Souza",
         'Charlie Kirk']

- Goodreads

In [63]:
#List of name in the format for Goodreads website
new_lista = []
for name in lista:
    new_lista.append(name.replace(' ','+'))
    

In [64]:
#Scraping all the existing quotes for the person I need from Goodreads
quotes = []
author = []

for name in new_lista:
    second_temp_quote = []
    second_temp_author = []
    r = requests.get("https://goodquotesapi.herokuapp.com/author/{}".format(name))
    data = r.json()
    if len(data) < 1:
        pass
    else:
        for page in tqdm_notebook(range(1,data['total_pages']+1)):
            new_r = requests.get("https://goodquotesapi.herokuapp.com/author/{}?page={}".format(name,page))
            new_data = new_r.json()
            first_temp_quote = []
            first_temp_author = []

            for i in range(20):
                temp_quote = []
                temp_author = []

                try: 
                    temp_quote.append(new_data['quotes'][i]['quote'])
                    temp_author.append(new_data['quotes'][0]['author'])

                except:
                    temp_quote.append(np.nan)
                    temp_author.append(np.nan)
                first_temp_author.append(temp_author)
                first_temp_quote.append(temp_quote)
            second_temp_quote.append(first_temp_quote)
            second_temp_author.append(first_temp_author)
    quotes.append(second_temp_quote)
    author.append(second_temp_author)

In [65]:
#Creating the goodreads Dataframe
goodreads = pd.DataFrame({
    'Author': [item for l in author for sublist in l for j in l for item in sublist], 
    'Quotes': [item for l in quotes for sublist in l for j in l for item in sublist],
    'source': 'goodreads'
})

In [66]:
#Leaving every cell in a string
goodreads.Author = goodreads.Author.apply(lambda x : x[0])
goodreads.Quotes = goodreads.Quotes.apply(lambda x : x[0])

In [67]:
#Deleting all the rows where the Author is not in the list
goodreads = goodreads[goodreads['Author'].isin(lista)]

- Wikiquote

In [68]:
#Loading the previous dataframe scraped from Wikiquotes
wikiquotes = pd.read_csv('../WikiQuote/right.csv')

In [70]:
wikiquotes = wikiquotes.drop('Political Orientation', axis=1)

In [71]:
wikiquotes['source'] = 'wikiquotes'

- Brainquote

In [72]:
#Creating a list for the Brainquote format
brain_lista = []
for n in lista:
    brain_lista.append(n.lower().replace(' ','_'))
    

In [73]:
brain_lista

['david_goodhart',
 'ben_shapiro',
 'jared_taylor',
 'david_duke',
 'matthew_goodwin',
 'eric_kaufmann',
 'richard_bertrand',
 'augustus_sol_invictus',
 'peter_hitchens',
 'david_irving',
 'jason_kessler',
 'sebastian_gorka',
 'tomislav_sunić',
 'paul_weyrich',
 'pat_buchanan',
 'steve_bannon',
 'ann_coulter',
 'milo_yiannopoulos',
 'vox_day',
 'steve_sailer',
 'stefan_molyneux',
 'alex_jones',
 'john_derbyshire',
 'mike_cernovich',
 'peter_brimelow',
 'katie_hopkins',
 'laura_loomer',
 'paul_joseph',
 'arthur_kemp',
 'tommy_robinson',
 'raheem_kassam',
 'jerome_corsi',
 'daniel_drezner',
 'lou_dobbs',
 'pamela_geller',
 'robert_spencer',
 'karl_hess',
 'hans-hermann_hoppe',
 'william_luther_pierce',
 'joel_pollak',
 'matt_drudge',
 'nicholas_wade',
 'john_r._bolton',
 "dinesh_d'souza",
 'charlie_kirk']

In [74]:
#Scraping the quotes from Brainquote website
q = []
a = []

for name in brain_lista:
    temp_quote = []
    temp_author = []
    r = requests.get("https://www.brainyquote.com/authors/{}".format(name))
    soup = BeautifulSoup(r.text,'html.parser')
    for each in soup.find_all('a', attrs={'title':"view quote"}):
        temp_quote.append(each.text)
        #if len(temp_quote) > 0:
        temp_author.append(name)
    q.append(temp_quote)
    a.append(temp_author)
    


In [75]:
#Creating the Brainquote dataframe
brain = pd.DataFrame({'Author': [item for l in a for item in l],
                      'Quotes': [item for l in q for item in l],
                      'source': 'brainquote'})

In [140]:
#Removing '_' from the name and capitalize Name and Surname
brain.Author = brain.Author.apply(lambda x : x.replace('_',' ').title())

In [144]:
Right_quotes = Right_quotes.drop_duplicates(subset='Quotes')

In [145]:
Right_quotes.shape

(2585, 3)

In [146]:
Right_quotes = Right_quotes.dropna()

- Detecting the language

In [147]:
language = []
for text in Right_quotes.Quotes:
    try:
        language.append(detect(text))
    except:
        language.append(np.nan)


In [148]:
Right_quotes['language'] = language

In [149]:
Right_quotes.language.value_counts()

en    2556
de       5
fr       3
no       3
it       3
af       2
tl       2
ca       1
cs       1
hu       1
fi       1
id       1
nl       1
es       1
pt       1
so       1
Name: language, dtype: int64

In [150]:
Right_quotes = Right_quotes[Right_quotes.language == 'en']

- Looking for quote in the quote

In [151]:
dup = {}
#count = 0
for name in Right_quotes.Author.unique():
    conta = []
    for text in Right_quotes[Right_quotes.Author==name].Quotes :
        count = 0
        for i in range(len(Right_quotes[Right_quotes.Author==name])):
            if text in Right_quotes[Right_quotes.Author==name].iloc[i,1]:
                count += 1


        conta.append(count)
    dup[name] = conta

In [152]:
Right_quotes['Counter'] = 0
for name in Right_quotes.Author.unique():
    print(name, ' done')
    Right_quotes.loc[Right_quotes.Author==name, 'Counter'] = dup[name]


Ben Shapiro  done
Jared Taylor  done
Peter Hitchens  done
David Irving  done
Ann Coulter  done
Milo Yiannopoulos  done
Pamela Geller  done
Vox Day  done
Stefan Molyneux  done
John Derbyshire  done
Mike Cernovich  done
Robert Spencer  done
Hans-Hermann Hoppe  done
Dinesh D'Souza  done
Augustus Sol Invictus  done
Paul Weyrich  done
Pat Buchanan  done
Steve Bannon  done
Alex Jones  done
Peter Brimelow  done
Katie Hopkins  done
Jerome Corsi  done
Lou Dobbs  done
Matt Drudge  done
Jason Kessler  done
Sebastian Gorka  done
Tomislav Sunić  done
David Duke  done
Richard Bertrand Spencer  done
Steve Sailer  done
Arthur Kemp  done
Tommy Robinson  done
Raheem Kassam  done
Ted Malloch  done
Daniel Drezner  done
Stephen Miller  done
Tucker Carlson  done
Karl Hess  done
William Luther Pierce  done
Joel Pollak  done
Nicholas Wade  done
John R. Bolton  done


In [154]:
Right_quotes = Right_quotes[Right_quotes.Counter==1]

In [155]:
Right_quotes.drop(['source', 'language', 'Counter'], axis=1, inplace=True)

In [156]:
#Adding the political orientation
Right_quotes['Political Orientation'] = 'Right'

In [160]:
Right_quotes.to_csv('right_quotes.csv', index=False)

In [161]:
#Selecting 4 Authors for a total of bit more than 100 quotes for the final test df
list_test = ['Steve Bannon', 'Katie Hopkins', 'Paul Weyrich', 'David Duke']

In [162]:
#Creating the Dataframe for the test
R_test = Right_quotes[Right_quotes['Author'].isin(list_test)]


In [163]:
R_test.Author.value_counts()


Steve Bannon     36
Katie Hopkins    31
Paul Weyrich     28
David Duke       16
Name: Author, dtype: int64

In [164]:
#removing the 4 authors from the main database
Right_quotes = Right_quotes[~Right_quotes['Author'].isin(list_test)]

In [168]:
#csv file for the train
Right_quotes.to_csv('train_right.csv')

In [167]:
#csv file for the test
R_test.to_csv('test_right.csv')

**SECOND PART**

In [4]:
#Loading Part

In [106]:
R_train = pd.read_csv('train_right.csv')

In [107]:
R_test = pd.read_csv('test_right.csv')

- Looking for quote about the author and not by them

In [109]:
def find_autoref(df):
    autoref = {}
    for name in df.Author.unique():
        conta = []

        for text in df[df.Author==name].Quotes :
            count = 0

            if name in text or name.split(' ')[-1] in text:
                    count += 1

            conta.append(count)
        autoref[name] = conta 
    return autoref

In [110]:
autoref = find_autoref(R_train)

In [111]:
R_train['auto'] = 0
for name in R_train.Author.unique():
    print(name, ' done')
    R_train.loc[R_train.Author==name, 'auto'] = autoref[name]

Ben Shapiro  done
Jared Taylor  done
Peter Hitchens  done
David Irving  done
Ann Coulter  done
Milo Yiannopoulos  done
Pamela Geller  done
Vox Day  done
Stefan Molyneux  done
John Derbyshire  done
Mike Cernovich  done
Robert Spencer  done
Hans-Hermann Hoppe  done
Dinesh D'Souza  done
Augustus Sol Invictus  done
Pat Buchanan  done
Alex Jones  done
Peter Brimelow  done
Jerome Corsi  done
Lou Dobbs  done
Matt Drudge  done
Jason Kessler  done
Sebastian Gorka  done
Tomislav Sunić  done
Richard Bertrand Spencer  done
Steve Sailer  done
Arthur Kemp  done
Tommy Robinson  done
Raheem Kassam  done
Ted Malloch  done
Daniel Drezner  done
Stephen Miller  done
Tucker Carlson  done
Karl Hess  done
William Luther Pierce  done
Joel Pollak  done
Nicholas Wade  done
John R. Bolton  done


In [112]:
R_train.auto.sum()

32

In [113]:
R_train=R_train[R_train.auto==0]

In [114]:
#Changing the name
R_train=R_train[R_train.Author!='Tucker Carlson']

In [115]:
#Changing the name
sol = R_train[R_train.Author=='Augustus Sol Invictus']

In [116]:
R_train=R_train[R_train.Author!='Augustus Sol Invictus']

In [118]:
from textacy.preprocess import preprocess_text
import re
#import nltk

In [119]:
nans = lambda df: df[df.isnull().any(axis=1)]
nans(R_train)

,Unnamed: 0,Author,Quotes,Political Orientation,auto


In [128]:
R_train['texicity'] = R_train.Quotes.apply(lambda x : preprocess_text(x, 
                                                                      lowercase=True, 
                                                                      no_punct=True, 
                                                                      no_accents=True, 
                                                                      no_contractions=True
                                                                     ))

In [129]:
R_train.head()

,Unnamed: 0,Author,Quotes,Political Orientation,auto,texicity
0,0,Ben Shapiro,Facts don't care about your feelings.,Right,0,facts do not care about your feelings
1,2,Ben Shapiro,"Without a clear moral vision, we devolve into moral relativism, and from there, into oblivion.",Right,0,without a clear moral vision we devolve into moral relativism and from there into oblivion
2,3,Ben Shapiro,There is no such thing as 'your truth'. There is the truth and your opinion.,Right,0,there is no such thing as your truth there is the truth and your opinion
3,4,Ben Shapiro,"Tolerance fails as a virtue, first of all, because it is in some ways demeaning to people. It is much better to speak of respect or empathy. But that is precisely the problem",Right,0,tolerance fails as a virtue first of all because it is in some ways demeaning to people it is much better to speak of respect or empathy but that is precisely the problem
4,5,Ben Shapiro,A man may be judged by his standard of entertainment as easily as by the standard of his work.,Right,0,a man may be judged by his standard of entertainment as easily as by the standard of his work


In [130]:
from nltk.corpus import stopwords


In [131]:
stops = stopwords.words('english')

In [133]:
def tok_stop(df):
    df['texicity'] = df['texicity'].apply(lambda x: x.split())
    df['texicity'] = df['texicity'].apply(lambda x : [item for item in x if item not in stops])
    df['texicity'] = df['texicity'].apply(lambda x: ' '.join(x))
    return df

In [134]:
R_train = tok_stop(R_train)

- Looking for quote in the quote with tokens

In [135]:
fake = {}
#count = 0
for name in R_train.Author.unique():
    print(name)
    conta = []
    for j,text in enumerate(R_train[R_train.Author==name].texicity):
        print(j)
        count = 0
        for i in range(len(R_train[R_train.Author==name])):
            if text in R_train[R_train.Author==name].iloc[i,5]:
                count += 1


        conta.append(count)
    fake[name] = conta

Ben Shapiro
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
Jared Taylor
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
Peter Hitchens
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
David Ir

In [136]:
R_train['rep'] = 0
for name in R_train.Author.unique():
    print(name, ' done')
    R_train.loc[R_train.Author==name, 'rep'] = fake[name]


Ben Shapiro  done
Jared Taylor  done
Peter Hitchens  done
David Irving  done
Ann Coulter  done
Milo Yiannopoulos  done
Pamela Geller  done
Vox Day  done
Stefan Molyneux  done
John Derbyshire  done
Mike Cernovich  done
Robert Spencer  done
Hans-Hermann Hoppe  done
Dinesh D'Souza  done
Pat Buchanan  done
Alex Jones  done
Peter Brimelow  done
Jerome Corsi  done
Lou Dobbs  done
Matt Drudge  done
Jason Kessler  done
Sebastian Gorka  done
Tomislav Sunić  done
Richard Bertrand Spencer  done
Steve Sailer  done
Arthur Kemp  done
Tommy Robinson  done
Raheem Kassam  done
Ted Malloch  done
Daniel Drezner  done
Stephen Miller  done
Karl Hess  done
William Luther Pierce  done
Joel Pollak  done
Nicholas Wade  done
John R. Bolton  done


In [139]:
R_train.to_csv('R_train_clean.csv', index=False)

- Dropping an Author that was in the test dataset

In [140]:
#Drop Katie Hopkins....she speak too much about herself
R_test=R_test[R_test.Author!='Katie Hopkins']

In [141]:
R_test=pd.concat([R_test, sol])

/Users/enrico/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


- Looking for quote about the authors and not by them

In [143]:
autoref_test=find_autoref(R_test)

In [144]:
R_test['auto'] = 0
for name in R_test.Author.unique():
    print(name, ' done')
    R_test.loc[R_test.Author==name, 'auto'] = autoref_test[name]

Paul Weyrich  done
Steve Bannon  done
David Duke  done
Augustus Sol Invictus  done


In [145]:
R_test['texicity'] = R_test.Quotes.apply(lambda x : preprocess_text(x, 
                                                                      lowercase=True, 
                                                                      no_punct=True, 
                                                                      no_accents=True, 
                                                                      no_contractions=True))

In [146]:
R_test=tok_stop(R_test)

- Quote in the quotes in the test dataset

In [148]:
fake = {}
#count = 0
for name in R_test.Author.unique():
    print(name)
    conta = []
    for i,text in enumerate(R_test[R_test.Author==name].texicity) :
        print(i)
        count = 0
        for i in range(len(R_test[R_test.Author==name])):
            if text in R_test[R_test.Author==name].iloc[i,5]:
                count += 1


        conta.append(count)
    fake[name] = conta

Paul Weyrich
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
Steve Bannon
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
David Duke
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Augustus Sol Invictus
0
1
2
3
4
5
6
7
8
9


In [149]:
R_test['rep'] = 0
for name in R_test.Author.unique():
    print(name, ' done')
    R_test.loc[R_test.Author==name, 'rep'] = fake[name]


Paul Weyrich  done
Steve Bannon  done
David Duke  done
Augustus Sol Invictus  done


In [151]:
R_test=R_test[R_test.rep==1]

In [152]:
R_test.to_csv('R_test_clean.csv', index=False)

**Third**

In [154]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

In [157]:
from sklearn.feature_extraction import text 

stop_words = text.ENGLISH_STOP_WORDS

In [161]:
stoppe = list(stop_words)

In [ ]:
def tok_stop(df):
    df['texicity'] = df['texicity'].apply(lambda x: x.split())
    df['texicity'] = df['texicity'].apply(lambda x : [item for item in x if item not in stoppe])
    df['texicity'] = df['texicity'].apply(lambda x: ' '.join(x))
    return df

In [162]:
R_train=tok_stop(R_train)


In [178]:
def bag_word(df, orientation, mm, mx, col):
    for i in range(mm,mx):
        vect = CountVectorizer(stop_words='english', ngram_range=(i,i))
        words = {}
        lefty = df[df['Political Orientation']==orientation]
        Left_Voc = ''.join(lefty[col])
        lefty_n_grams = vect.build_analyzer()(Left_Voc)
        words['Left'] = Counter(lefty_n_grams).most_common()
        print('Number of gram: ', i)
        print(Counter(lefty_n_grams).most_common())
        print()
        print()
        return Counter(lefty_n_grams).most_common(100)

- Looking for quote in the quote using the n_grams from countvectorizer

In [170]:
list_ind=[]
for gram in list_gram:
    if gram[1]>1:
        try:
            (one,two)=(R_train.loc[R_train['texicity'].str.contains(gram[0]), 'texicity'].index)
            if len(R_train.loc[one,'texicity'])>len(R_train.loc[two,'texicity']):
                list_ind.append(two)
            else:
                list_ind.append(one)

        except:
            pass

In [1]:
l=list(set(list_ind))


In [174]:
R_train=R_train.drop(l)

- Removing the last bunch manually

In [194]:
l_2=[297, 777, 1066, 1526, 1536,380, 387, 414, 526, 527, 528, 538, 534, 540, 541, 93, 126, 
     6, 144]

In [195]:
R_train=R_train.drop(l_2)

In [203]:
R_train.to_csv('R_train_clean.csv')